**Mount drive and clone repository**

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

# Clone repository
import sys
!git clone https://github.com/fgiacome/MLDL23-FL-project.git
sys.path.append('./MLDL23-FL-project/')
%cd MLDL23-FL-project
!git checkout fed_w_avg

**Transforms and model importations**

In [ ]:
# Importations
import main
import torch
import random
from datasets.idda import IDDADataset
from models import deeplabv3, mobilenetv2
import datasets.ss_transforms as sstr

# Fix random seed
random.seed(300890)

# args for dataset and model importation
class args:
  dataset = 'idda'
  model = 'deeplabv3_mobilenetv2'

# Model
model = main.model_init(args).cuda()

# Transforms
train_transforms = sstr.Compose([
            sstr.ColorJitter(brightness = (0.55, 1.6), contrast = (0.6, 1.6),
                             saturation = (0.5, 1.6), hue = (-0.05, 0.05)),
            sstr.RandomRotation(degrees = (-5, 5)),
            sstr.PadCenterCrop((925, 1644), fill=255), # this removes the
                                                       # rotation margins
            sstr.Resize((512, 928)),
            sstr.ToTensor(),
            sstr.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
test_transforms = sstr.Compose([
            sstr.Resize((512, 928)),
            sstr.ToTensor(),
            sstr.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
train_transforms_degrade = []
train_transforms_degrade.append(sstr.Compose([
            sstr.ColorJitter(brightness = (0.55, 1.6), contrast = (0.6, 1.6),
                             saturation = (0.5, 1.6), hue = (-0.05, 0.05)),
            sstr.RandomRotation(degrees = (-5, 5)),
            sstr.PadCenterCrop((925, 1644), fill=255), # this removes the
                                                       # rotation margins
            sstr.Resize((512, 928)),
            sstr.DegradeGaussianBlur(),
            sstr.ToTensor(),
            sstr.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]))
train_transforms_degrade.append(sstr.Compose([
            sstr.ColorJitter(brightness = (0.55, 1.6), contrast = (0.6, 1.6),
                             saturation = (0.5, 1.6), hue = (-0.05, 0.05)),
            sstr.RandomRotation(degrees = (-5, 5)),
            sstr.PadCenterCrop((925, 1644), fill=255), # this removes the
                                                       # rotation margins
            sstr.Resize((512, 928)),
            sstr.DegradeGaussianNoise(),
            sstr.ToTensor(),
            sstr.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]))
train_transforms_degrade.append(sstr.Compose([
            sstr.ColorJitter(brightness = (0.55, 1.6), contrast = (0.6, 1.6),
                             saturation = (0.5, 1.6), hue = (-0.05, 0.05)),
            sstr.RandomRotation(degrees = (-5, 5)),
            sstr.PadCenterCrop((925, 1644), fill=255), # this removes the
                                                       # rotation margins
            sstr.Resize((512, 928)),
            sstr.DegradePumpGreen(),
            sstr.ToTensor(),
            sstr.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]))

**Train, Test Same Dom, Test Diff Dom importations**

In [ ]:
# Importations
import os
import json
from collections import OrderedDict

# Initialization of clients datasets
root = 'data/idda'
with open(os.path.join(root, 'train.json'), 'r') as f:
  clients_dict = json.load(f)

# Dict -> datasets
train_clients_df = OrderedDict()
map_clients_transforms = {18: 0, 19: 0, 20: 1, 21: 1, 22: 2, 23: 2}
for i, client_id in enumerate(clients_dict.keys()):
  if i < 18:
    train_clients_df[client_id] = IDDADataset(root = root, list_samples = clients_dict[client_id],
                                             transform = train_transforms, client_name = client_id)
  else:
    train_clients_df[client_id] = IDDADataset(root = root, list_samples = clients_dict[client_id],
                                             transform = train_transforms_degrade[map_clients_transforms[i]], client_name = client_id)
train_clients_names = list(train_clients_df.keys())

# Test set
with open(os.path.join(root, 'test_same_dom.txt'), 'r') as f:
    test_same_dom_data = f.read().splitlines()
    test_same_dom_df = IDDADataset(root = root, list_samples = test_same_dom_data,
                                        transform = test_transforms,
                                        client_name='test_same_dom')

with open(os.path.join(root, 'test_diff_dom.txt'), 'r') as f:
    test_diff_dom_data = f.read().splitlines()
    test_diff_dom_df = IDDADataset(root=root, list_samples = test_diff_dom_data,
                                        transform = test_transforms,
                                        client_name='test_diff_dom')

**Combinations**

In [ ]:
# Combinations to test with the format [n_clients_round, n_local_epochs]
combinations = {'comb_2_1': [2, 1],
                'comb_2_3': [2, 3],
                'comb_2_6': [2, 6],
                'comb_4_1': [4, 1],
                'comb_4_3': [4, 3],
                'comb_4_6': [4, 6],
                'comb_8_1': [8, 1],
                'comb_8_2': [8, 3],
                'comb_8_6': [8, 6]}

**Test cell**

In [ ]:
# Client and Server importationsù
import copy
import client
import server as svr

# Select combination
################################################################################
############################## CHOOSE COMB! ####################################
################################################################################
comb = 'comb_4_6' ##############################################################
################################################################################
############################## CHOOSE COMB! ####################################
################################################################################
################################################################################
# Comb_dict formats: [<n_clients_round>, <n_local_epochs>] #####################
#################### [       0     ,             1       ] #####################
################################################################################

# Assign combinations
clients_per_round = combinations[comb][0]
epochs_per_round = combinations[comb][1]

# Number of total rounds
num_rounds = 20

# !!CHOOSE REDUCTION!!
# reduction \in \{'MeanReduction, HardNegativeMining'\}
reduction = 'MeanReduction'

# Clients initializations
# - Train clients
train_clients = [client.Client(client_id = name, client_dataset = train_clients_df[name], batch_size = 8,
                 model = main.model_init(args).cuda(), dataloader = 'train',
                 optimizer = 'Adam', lr = 1e-3, device = 'cuda:0',
                 reduction = reduction) for name in train_clients_names]

# - Test clients [same_dom, diff_dom]
test_clients = [client.Client(client_id = "test_same_dom", client_dataset = test_same_dom_df, batch_size = 16,
                model = main.model_init(args).cuda(), dataloader = 'test',
                device = 'cuda:0', reduction = reduction),
                client.Client(client_id = "test_diff_dom", client_dataset = test_diff_dom_df, batch_size = 16,
                model = main.model_init(args).cuda(), dataloader = 'test',
                device = 'cuda:0', reduction = reduction)]

# Server initialization
server = svr.Server(clients_per_round = clients_per_round, num_rounds = num_rounds,
                epochs_per_round = epochs_per_round, train_clients = train_clients,
                test_clients = test_clients, model = main.model_init(args).cuda(),
                use_prior = False, n_rounds_no_prior = 1, random_state = 300890)

# Train server on selected combination
results = server.train("/content/drive/MyDrive/mldl-project-2b/Checkpoints5.1-noprior/model_checkpoint_round")
results_eval = server.eval_train()
results_test = server.test()
results_weights_track = server.weights_track.copy()

# Cast dict
# Results dict format {<round_index>: <results>}
results_complete = {
      "Train" : results,
      "Eval" : results_eval,
      "Test" : results_test,
      "Weights": results_weights_track.copy()
  }

# Save results
# Define file name
filename = 'noprior_results_' + comb + '.json'
path = '/content/drive/MyDrive/mldl-project-2b/Results5/'

# Save data on file system (Remember to download it!)
with open(path + filename, 'w') as fp:
  json.dump(results_complete, fp)

**Save results**